<a href="https://colab.research.google.com/github/taekg/AGV_Project/blob/main/RoadFollowingFeedback.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 5장. Road Following Feedback
이전 챕터에서 우리는 직접 제작한 model 을 활용하여 Road Following을 진행하였습니다.  
원활하게 동작하는 구간도 있고, 예상 외로 동작하는 구간도 발견할 수 있을 겁니다.  
이번 챕터에서는 수동으로 인공지능 무인운반차량(AGV)을 움직여보면서, 실제 학습한 model이 구간에서 어떤 방향으로 가고자 하는 지를 살펴보며, 보강할 필요가 있는 데이터 셋을 찾는 코드를 실습해봅니다.  

## 라이브러리 가져오기

In [ ]:
import torchvision
import torch
import torchvision.transforms as transforms
import torch.nn.functional as F
import cv2
import PIL.Image
import numpy as np

from IPython.display import display
import ipywidgets
import ipywidgets.widgets as widgets
import traitlets
from jetbot import Robot, Camera, bgr8_to_jpeg

import threading
import time

## model 파일 불러오기

In [ ]:
model = torchvision.models.resnet18(pretrained=False)
model.fc = torch.nn.Linear(512, 2)
model.load_state_dict(torch.load('best_steering_model_xy_test.pth'))

device = torch.device('cuda')
model = model.to(device)
model = model.eval().half()

mean = torch.Tensor([0.485, 0.456, 0.406]).cuda().half()
std = torch.Tensor([0.229, 0.224, 0.225]).cuda().half()
print('model load success')

## robot, camera 객체 생성하기

In [ ]:
robot = Robot()
camera = Camera()

## 위젯 코드 추가하기

In [ ]:
#display code

roadlbl = ipywidgets.Label(value="Road Recog")
image_widget = widgets.Image(format='jpeg', width=224, height=224)
x_slider = widgets.FloatSlider(min=-1.0, max=1.0, step=0.001, description='x')
y_slider = ipywidgets.FloatSlider(min=0, max=1.0, orientation='vertical', description='y')
hbox = widgets.HBox([image_widget, y_slider])
vbox = widgets.VBox([roadlbl, hbox, x_slider])

# create buttons
button_layout = widgets.Layout(width='100px', height='80px', align_self='center')
stop_button = widgets.Button(description='stop', button_style='danger', layout=button_layout)
forward_button = widgets.Button(description='forward', layout=button_layout)
backward_button = widgets.Button(description='backward', layout=button_layout)
left_button = widgets.Button(description='left', layout=button_layout)
right_button = widgets.Button(description='right', layout=button_layout)
# 레이아웃 생성 후,버튼 5개 생성

# display buttons
middle_box = widgets.HBox([left_button, stop_button, right_button], layout=widgets.Layout(align_self='center'))
controls_box = widgets.VBox([forward_button, middle_box, backward_button])

hbox2 = widgets.HBox([vbox, controls_box])
display(hbox2)

## 버튼에 대응할 함수 생성과 연결하기

In [ ]:
def stop(change):
    robot.stop()

def step_forward(change):
    robot.forward(0.4)

def step_backward(change):
    robot.backward(0.4)

def step_left(change):
    robot.left(0.3)
    time.sleep(0.5)
    robot.stop()

def step_right(change):
    robot.right(0.3)
    time.sleep(0.5)
    robot.stop()

stop_button.on_click(stop)
forward_button.on_click(step_forward)
backward_button.on_click(step_backward)
left_button.on_click(step_left)
right_button.on_click(step_right)

## 전처리 함수 정의하기
이전 챕터와 마찬가지로, 전처리 함수를 정의합니다.  
이미지에 model이 가고자 하는 방향을 원과 직선을 이용하여 표시합니다.

In [ ]:
def preprocess(imageInput):
    image = PIL.Image.fromarray(imageInput)
    image = transforms.functional.to_tensor(image).to(device).half()
    image.sub_(mean[:, None, None]).div_(std[:, None, None])
    return image[None, ...]

th_flag = True
def display_xy():
    while th_flag:
        image = camera.value
        xy = model(preprocess(image)).detach().float().cpu().numpy().flatten()
        x = xy[0]
        y = (0.5 - xy[1]) / 2.0

        x_slider.value = x
        y_slider.value = y

        x = int(x * 224 / 2 + 112)
        y = int(y * 224 / 2 + 112)
        image = cv2.circle(image, (x, y), 8, (0, 255, 0), 3)
        image = cv2.circle(image, (112, 224), 8, (0, 0,255), 3)
        image = cv2.line(image, (x,y), (112,224), (255,0,0), 3)
        image_widget.value = bgr8_to_jpeg(image)
        time.sleep(0.1)

## image_thread 객체 생성하기
객체를 생성하고 start() 를 하면, 카메라로 촬영한 길을 보고 가야할 방향을 이미지에 표시해서 송출합니다.  
예상 외로 방향이 다르게 표시되는 구간을 찾아서 데이터를 재수집합니다.  
데이터를 재수집하고 나서, 다시 model을 훈련시켜서 live demo까지 진행합니다.  

In [ ]:
image_thread = threading.Thread(target = display_xy)
image_thread.start()

실행 결과  





## 프로젝트 종료하기

In [ ]:
camera.stop()
robot.stop()
th_flag = False
image_thread.join()

이번 챕터에서 Road Following 을 원활히 할 수 있도록 model을 확실히 학습시키도록 합니다.  
다음 챕터에서는 Jupyter Widget 에 대해서 자세히 학습하고, Firebase DB 세팅하는 법을 실습합니다.